In [1]:
import os
import json
import random
import itertools
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model

# Define dataset paths
base_dir = "/kaggle/input/the-wildfire-dataset/the_wildfire_dataset_2n_version"
train_dir = f"{base_dir}/train"
val_dir = f"{base_dir}/val"
test_dir = f"{base_dir}/test"

IMG_SIZE = (128, 128)
BATCH_SIZE = 16
SEED = 1234

# Load datasets correctly
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir, image_size=IMG_SIZE, batch_size=BATCH_SIZE, label_mode="binary", seed=SEED
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir, image_size=IMG_SIZE, batch_size=BATCH_SIZE, label_mode="binary", seed=SEED
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir, image_size=IMG_SIZE, batch_size=BATCH_SIZE, label_mode="binary", seed=SEED
)

# Define normalization layer
normalization_layer = tf.keras.layers.Rescaling(1.0 / 255)

# Apply normalization before batching
def preprocess_ds(dataset):
    dataset = dataset.map(lambda x, y: (normalization_layer(x), y))
    return dataset.prefetch(tf.data.AUTOTUNE)  # Avoids bottlenecks

# Apply preprocessing
train_ds = preprocess_ds(train_ds)
val_ds = preprocess_ds(val_ds)
test_ds = preprocess_ds(test_ds)

# Define hyperparameter search space
HYPERPARAMS = {
    "lr": [0.0001, 0.001, 0.01],
    "bs": [32, 64],
    "drop": [0.2, 0.5],
    "filters": [32, 64],
    "kernel": [(3, 3), (5, 5)],
    "dense": [128, 256],
    "aug": [True, False],
    "act": ["relu", "leaky_relu"],
}

EPOCHS = 10  
INPUT_SHAPE = (128, 128, 3)

# Generate all possible hyperparameter combinations
all_combinations = list(itertools.product(*HYPERPARAMS.values()))
N_RANDOM_TRIALS = min(10, len(all_combinations))  
random_combinations = random.sample(all_combinations, N_RANDOM_TRIALS)

# Function to create CNN model
def create_cnn_model(lr, drop, filters, kernel, dense, act, aug=None):
    """Creates and compiles a CNN model with flexible hyperparameters."""
    inputs = keras.Input(shape=INPUT_SHAPE)

    # Apply augmentation before convolution layers if enabled
    if aug:
        x = aug(inputs)
    else:
        x = inputs

    # Convolutional layers
    x = layers.Conv2D(filters, kernel, activation=act)(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(filters * 2, kernel, activation=act)(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(filters * 4, kernel, activation=act)(x)
    x = layers.MaxPooling2D()(x)

    # Flatten and fully connected layers
    x = layers.Flatten()(x)
    x = layers.Dropout(drop)(x)
    x = layers.Dense(dense, activation=act)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )

    return model

# Directory to save models
save_dir = "saved_models"
os.makedirs(save_dir, exist_ok=True)

best_val_acc = 0
best_model_info = {}
results = []  # Store results for all hyperparameter combinations

# Training loop
for i, hyperparams in enumerate(random_combinations, start=1):
    print(f"\nTraining {i}/{N_RANDOM_TRIALS}: {hyperparams}...")

    # Unpack hyperparameters
    lr, bs, drop, filters, kernel, dense, aug, act = hyperparams

    # Fix LeakyReLU issue
    if act == "leaky_relu":
        act = layers.LeakyReLU(negative_slope=0.1)

    # Define data augmentation if enabled
    data_augmentation = (
        keras.Sequential([
            layers.RandomFlip("horizontal"),
            layers.RandomRotation(0.1),
            layers.RandomZoom(0.1),
            layers.Resizing(128, 128)  # Ensures augmentation does not change image size
        ])
        if aug else None
    )

    # Create the CNN model
    model = create_cnn_model(lr, drop, filters, kernel, dense, act, data_augmentation)

    # Apply augmentation correctly
    if aug:
        train_ds_aug = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))
    else:
        train_ds_aug = train_ds

    # **EARLY STOPPING CALLBACK**
    early_stopping = keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=2, restore_best_weights=True)

    # Train the model with no logs (verbose=0)
    history = model.fit(
        train_ds_aug,  
        validation_data=val_ds,
        epochs=EPOCHS,
        callbacks=[early_stopping],
        verbose=0  # Hide training logs
    )

    # Find the epoch where early stopping triggered
    stopped_epoch = len(history.history["accuracy"])

    print(f"  Early stopping triggered at epoch {stopped_epoch}.")

    # Evaluate on test dataset
    test_loss, test_acc = model.evaluate(test_ds, verbose=0)

    # Store results for this combination
    model_info = {
        "Combination": i,
        "Learning Rate": lr,
        "Batch Size": bs,
        "Dropout": drop,
        "Filters": filters,
        "Kernel": kernel,
        "Dense": dense,
        "Augmentation": aug,
        "Activation": str(act),
        "Train Accuracy": history.history["accuracy"][-1],
        "Val Accuracy": history.history["val_accuracy"][-1],
        "Test Accuracy": test_acc,
    }
    results.append(model_info)

    # Track the best model
    if model_info["Val Accuracy"] > best_val_acc:
        best_val_acc = model_info["Val Accuracy"]
        best_model_info = model_info

        # Save the best model
        best_model_path = os.path.join(save_dir, "best_model.keras")
        model.save(best_model_path)

        # Save training history
        best_history_path = os.path.join(save_dir, "best_history.json")
        with open(best_history_path, "w") as f:
            json.dump(history.history, f)

        print(f"  New best model saved: {best_model_path}")

# Save all results
results_path = os.path.join(save_dir, "all_results.json")
with open(results_path, "w") as f:
    json.dump(results, f)

# Save best model info
best_info_path = os.path.join(save_dir, "best_model_info.json")
with open(best_info_path, "w") as f:
    json.dump(best_model_info, f)

print(f"\nAll results saved to: {results_path}")
print(f"Best model details saved to: {best_info_path}")
print(f"Best model file: {best_model_path}")
print(f"Best training history file: {best_history_path}")

Found 1887 files belonging to 2 classes.
Found 402 files belonging to 2 classes.
Found 410 files belonging to 2 classes.

Training 1/10: (0.001, 64, 0.2, 32, (5, 5), 256, True, 'leaky_relu')...
  Early stopping triggered at epoch 4.
  New best model saved: saved_models/best_model.keras

Training 2/10: (0.001, 64, 0.5, 64, (5, 5), 128, False, 'relu')...
  Early stopping triggered at epoch 10.
  New best model saved: saved_models/best_model.keras

Training 3/10: (0.01, 64, 0.2, 64, (5, 5), 128, False, 'leaky_relu')...
  Early stopping triggered at epoch 3.

Training 4/10: (0.01, 32, 0.5, 64, (5, 5), 256, False, 'leaky_relu')...
  Early stopping triggered at epoch 3.

Training 5/10: (0.001, 64, 0.2, 32, (3, 3), 128, False, 'relu')...
  Early stopping triggered at epoch 7.
  New best model saved: saved_models/best_model.keras

Training 6/10: (0.0001, 32, 0.5, 64, (5, 5), 256, True, 'relu')...
  Early stopping triggered at epoch 5.

Training 7/10: (0.01, 64, 0.5, 64, (5, 5), 128, False, 're